In [ ]:
#| echo: false
#| include: false
import sys
from pathlib import Path
sys.path.append(str(Path("..") / "Code"))
from svm_source import *

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

from sklearn import svm
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from time import time

scaler = StandardScaler()

import warnings
warnings.filterwarnings("ignore")

plt.style.use('ggplot')

## Introduction
Les machines à vecteurs de support (Support Vector Machine, SVM, en anglais), sont un ensemble de méthodes d'apprentissage supervisé utilisées pour la classification, la régression et la détection des valeurs aberrantes. La popularité des méthodes SVM, pour la classification binaire en particulier, provient du fait qu’elles reposent sur l’application d’algorithmes de recherche de règles de décision linéaires : on parle d’hyperplans (affines) séparateurs.

## Question 1
Pour commencer, écrivons un code qui va classifier la classe 1 contre la classe 2 du dataset $\texttt{iris}$ en utilisant les deux premières variables et un noyau linéaire. 


In [ ]:
iris = datasets.load_iris()
X = iris.data
X = scaler.fit_transform(X)
y = iris.target
X = X[y != 0, :2]
y = y[y != 0]

X, y = shuffle(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 200))}

clf_linear = GridSearchCV(SVC(),parameters,n_jobs=-1)

clf_linear.fit(X_train, y_train)


print('Generalization score for linear kernel: %s, %s' %
      (clf_linear.score(X_train, y_train),
       clf_linear.score(X_test, y_test)))

ici, commenter.

## Question 2

Nous voulons alors comparer ce résultat avec un SVM basé sur noyau polynomial.


In [ ]:
Cs = list(np.logspace(-3, 3, 5))
gammas = 10. ** np.arange(1, 2)
degrees = np.r_[1, 2, 3]



parameters = {'kernel': ['poly'], 'C': Cs, 'gamma': gammas, 'degree': degrees}


clf_poly = GridSearchCV(SVC(),parameters,n_jobs=-1)

clf_poly.fit(X_train,y_train)

print(clf_poly.best_params_)

print('Generalization score for polynomial kernel: %s, %s' %
      (clf_poly.score(X_train, y_train),
       clf_poly.score(X_test, y_test)))


def f_linear(xx):
    """Classifier: needed to avoid warning due to shape issues"""
    return clf_linear.predict(xx.reshape(1, -1))

def f_poly(xx):
    """Classifier: needed to avoid warning due to shape issues"""
    return clf_poly.predict(xx.reshape(1, -1))

plt.ion()
plt.figure(figsize=(15, 5))
plt.subplot(131)
plot_2d(X, y)
plt.title("iris dataset")

plt.subplot(132)
frontiere(f_linear, X, y)
plt.title("linear kernel")

plt.subplot(133)
frontiere(f_poly, X, y)

plt.title("polynomial kernel")
plt.tight_layout()
plt.draw()

surement faux ! comparer les résultats

# SVM GUI

## Question 3



# Classification de visages

Intéressons nous maintenant à un problème de classification de visages.  

## Question 4

Nous cherchons à montrer l'influence du paramètre de régularisation. Nous affichons dans la figure (METTRE REF) le score d'apprentissage en fonction de C sur une échelle logarithmique entre 1e5 et 1e-5(qui n'est rien d'autre que 1 - l'erreur de prédiction).


In [ ]:
#| echo: false
#| include: false

"""
The dataset used in this example is a preprocessed excerpt
of the "Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

  _LFW: http://vis-www.cs.umass.edu/lfw/
"""

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4,
                              color=True, funneled=False, slice_=None,
                              download_if_missing=True)

images = lfw_people.images
n_samples, h, w, n_colors = images.shape

target_names = lfw_people.target_names.tolist()

names = ['Tony Blair', 'Colin Powell']


idx0 = (lfw_people.target == target_names.index(names[0]))
idx1 = (lfw_people.target == target_names.index(names[1]))
images = np.r_[images[idx0], images[idx1]]
n_samples = images.shape[0]
y = np.r_[np.zeros(np.sum(idx0)), np.ones(np.sum(idx1))].astype(int)

plot_gallery(images, np.arange(12))
plt.show()

X = (np.mean(images, axis=3)).reshape(n_samples, -1)


X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

indices = np.random.permutation(X.shape[0])
train_idx, test_idx = indices[:X.shape[0] // 2], indices[X.shape[0] // 2:]
X_train, X_test = X[train_idx, :], X[test_idx, :]
y_train, y_test = y[train_idx], y[test_idx]
images_train, images_test = images[
    train_idx, :, :, :], images[test_idx, :, :, :]

In [ ]:
print("--- Linear kernel ---")
print("Fitting the classifier to the training set")
t0 = time()

# fit a classifier (linear) and test all the Cs
Cs = 10. ** np.arange(-5, 6)
scores = []
for C in Cs:

    clf = SVC(kernel='linear', C=C)
    clf.fit(X_train,y_train)

    score = clf.score(X_train, y_train)
    scores.append(score)

ind = np.argmax(scores)
print("Best C: {}".format(Cs[ind]))

plt.figure()
plt.plot(Cs, scores)
plt.xlabel("Parametres de regularisation C")
plt.ylabel("Scores d'apprentissage")
plt.xscale("log")
plt.tight_layout()
plt.show()
print("Best score: {}".format(np.max(scores)))

print("Predicting the people names on the testing set")

t0=time()
clf= SVC(kernel='linear', C=Cs[ind])
clf.fit(X_train,y_train)

# fin du fait par moi, vérifier avec les autres

print("done in %0.3fs" % (time() - t0))
# The chance level is the accuracy that will be reached when constantly predicting the majority class.
print("Chance level : %s" % max(np.mean(y), 1. - np.mean(y)))
print("Accuracy : %s" % clf.score(X_test, y_test))

y_pred = y_pred[:y_test.shape[0]]
prediction_titles = [title(y_pred[i], y_test[i], names)
                     for i in range(y_pred.shape[0])]

plot_gallery(images_test, prediction_titles)
plt.show()

####################################################################
# Look at the coefficients
plt.figure()
plt.imshow(np.reshape(clf.coef_, (h, w)))
plt.show()

COMMENTER

## Question 5


In [ ]:
def run_svm_cv(_X, _y):
    _indices = np.random.permutation(_X.shape[0])
    _train_idx, _test_idx = _indices[:_X.shape[0] // 2], _indices[_X.shape[0] // 2:]
    _X_train, _X_test = _X[_train_idx, :], _X[_test_idx, :]
    _y_train, _y_test = _y[_train_idx], _y[_test_idx]

    _parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 5))}
    _svr = svm.SVC()
    _clf_linear = GridSearchCV(_svr, _parameters)
    _clf_linear.fit(_X_train, _y_train)

    print('Generalization score for linear kernel: %s, %s \n' %
          (_clf_linear.score(_X_train, _y_train), _clf_linear.score(_X_test, _y_test)))

print("Score sans variable de nuisance")
# fait par moi
run_svm_cv(X, y)
# fin du fait par moi

print("Score avec variable de nuisance")
n_features = X.shape[1]
# On rajoute des variables de nuisances
sigma = 1
noise = sigma * np.random.randn(n_samples, 300, )
X_noisy = np.concatenate((X, noise), axis=1)
X_noisy = X_noisy[np.random.permutation(X.shape[0])]
# fait par moi
run_svm_cv(X_noisy, y)
# fin du fait par moi

## Question 6
